In [ ]:
from casadi import *
import matplotlib.pyplot as plt
import numpy as np
from matplotlib import animation, rc
from IPython.display import HTML

In [ ]:
def timecrossings(arr, times, pos):
    crossings = []
    sign_arr = sign(arr)
    for ii in range(1,len(arr)):
        if sign_arr[ii-1] != sign_arr[ii]:
            if cos(pos[ii]) > -0.99 :
                crossings.append((times[ii-1] + times[ii])/2)
    return crossings

In [ ]:
def plot_results(sol, max_par, N):
    arr = sol.value(X)
    margin_ang = np.arcsin(sol.value(u_m))
    timescale_x = np.linspace(0, sol.value(T), N+1)
    timescale_u = np.linspace(0, sol.value(T), N)
    arr_u = sol.value(U)
    plt.figure(figsize=[10,7])
    plt.plot(timescale_x,arr[:,0], label = '$x$')
    plt.plot(timescale_x,arr[:,1], label = "$x'$")
    plt.plot(timescale_u,arr_u[:], label = 'u')
    plt.plot(timescale_x,2+cos(arr[:,0]),':', label = '$2+cos(x)$')
    cross_points = timecrossings(arr[:,1], timescale_x, arr[:,0])
    plt.hlines([0,pi, -pi], 0, timescale_u[-1], 'k', 'dotted')
    plt.hlines([-max_par,max_par], 0, timescale_u[-1], 'r', 'dotted')
    plt.hlines([np.pi-margin_ang,-np.pi+margin_ang], 0, timescale_u[-1], 'g', 'dotted', label = "g_par = max par")
    plt.vlines(cross_points, -pi, pi, 'k', 'dotted')
    print(f'Max par: {max_par}, number of crossing points: {len(cross_points)}')
    plt.legend()

In [ ]:
def create_anim(sol):
    fig, ax = plt.subplots()

    fig.set_size_inches([8,8])
    ax.set_xlim(( -1.5, 1.5))
    ax.set_ylim(( -1.5, 1.5))

    circle2 = plt.Circle((0, 0), 1, color='b', ls = ":", fill=False)
    ax.add_artist(circle2)

    line, = ax.plot([], [], lw=2)
    point, = ax.plot([], [], marker='o', markersize=15, color="red")
    text = ax.text(0.2, 0, "", fontsize = 12)
    text_2 = ax.text(0.2, -0.15, "", fontsize = 12)
    text_3 = ax.text(0.2, -0.3, "", fontsize = 12)
    
    def init():
        line.set_data([], [])
        point.set_data([], [])
        text.set_text('')
        return (line,)
    def animate(i):
        x = [0, np.sin(sol.value(X)[i,0])]
        y = [0, -np.cos(sol.value(X)[i,0])]
        line.set_data(x, y)    
        point.set_data(x[1], y[1])
        text.set_text("U = %.6f" % sol.value(U)[i])
        text_2.set_text(r"$\dot{\theta}$" + " = %.6f" % sol.value(X)[i,1])
        text_3.set_text("par g = %.6f" % x[1])
        return (line,)
    anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)
    return anim

## Pendulum exercise
$$
\begin{split}\begin{array}{lc}
\begin{array}{l}
\text{minimize:} \\
x(\cdot) \in \mathbb{R}^2, \, u(\cdot) \in \mathbb{R}
\end{array}
\quad \displaystyle \int_{t=0}^{T}{cos(x_0) \, dt}
\\
\\
\text{subject to:} \\
\\
\begin{array}{ll}
\left\{
\begin{array}{l}
\dot{x}_0 = x_1 \\
\dot{x}_1 = u - sin(x_0) \\
-u_{max} \le u \le u_{max} , \quad
\end{array} \right. & \text{for} \, 0 \le t \le T \\
x_0(0)=0, \quad x_1(0)=0, x_0(T) = pi/2 , x_1(T) = 0
\end{array}
\end{array}\end{split}
$$
with $T=10$.

siendo $$x_0 = \theta$$ $$x_1 = \theta'$$

In [ ]:
x = MX.sym('x', 2)
t = MX.sym('t')
dt = MX.sym('dt')
u = MX.sym('u')

In [ ]:
rhs = vertcat(x[1], u-sin(x[0]))
#rhs = vertcat(x[1], u)
F = Function('F', [x, u], [rhs])

In [ ]:
k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

In [ ]:
new_x_expr = x + dt * F(x, u)
new_x = Function('New_x', [x, u, dt], [new_x_expr])

## Opti problem

In [ ]:
N = 200

In [ ]:
opti = Opti()
opti.solver('ipopt')

In [ ]:
X = opti.variable(N+1,2)
U = opti.variable(N)
#T = opti.variable()
T = opti.parameter()
u_m = opti.parameter()
#t_m = opti.parameter()

In [ ]:
cost = sum1(2+cos(X[:,0]))*T #**2
#cost = -sum1(X[:,0])
opti.minimize(cost)

In [ ]:
opti.subject_to(X[0,:].T == [0, 0])
opti.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti.subject_to(opti.bounded(-0.001,X[-1,1],0.001))

In [ ]:
for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti.set_initial(X[:,1], pi/N)
#opti.set_initial(T, 50)
opti.set_value(T, 40)
max_par = 0.09
opti.set_value(u_m, max_par)

In [ ]:
sol = opti.solve()

In [ ]:
plot_results(sol, max_par, N)

In [ ]:
anim = create_anim(sol)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
results = []
for ii in [0.11, 0.105, 0.1, 0.095, 0.09, 0.085]:
    opti.set_value(u_m, ii)
    est_t = (2 + sqrt(1/ii))*8
    #opti.set_initial(T, est_t)
    opti.set_value(T, 40)
    try:
        sol = opti.solve()
    except:
        pass
    else:
        results.append([sol, ii, N])

In [ ]:
for res in results:
    plot_results(*res)

## Animation

In [ ]:
fig, ax = plt.subplots()

fig.set_size_inches([8,8])
ax.set_xlim(( -1.5, 1.5))
ax.set_ylim(( -1.5, 1.5))

circle2 = plt.Circle((0, 0), 1, color='b', ls = ":", fill=False)
ax.add_artist(circle2)

line, = ax.plot([], [], lw=2)
point, = ax.plot([], [], marker='o', markersize=15, color="red")
text = ax.text(0.2, 0, "", fontsize = 12)
text_2 = ax.text(0.2, -0.15, "", fontsize = 12)
text_3 = ax.text(0.2, -0.3, "", fontsize = 12)

In [ ]:
def init():
    line.set_data([], [])
    point.set_data([], [])
    text.set_text('')
    return (line,)

In [ ]:
def animate(i):
    x = [0, np.sin(sol.value(X)[i,0])]
    y = [0, -np.cos(sol.value(X)[i,0])]
    line.set_data(x, y)    
    point.set_data(x[1], y[1])
    text.set_text("U = %.6f" % sol.value(U)[i])
    text_2.set_text(r"$\dot{\theta}$" + " = %.6f" % sol.value(X)[i,1])
    text_3.set_text("par g = %.6f" % x[1])
    return (line,)

In [ ]:
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=N, interval=20, 
                               blit=True)

In [ ]:
HTML(anim.to_jshtml())

## Integración de la acción para comprobación

In [ ]:
def euler_step(x, u, dt):
    return x + dt * F(x, u)

In [ ]:
def rk4_step(x, u, dt):
    k1 = F(x, u);
    k2 = F(x + dt/2 * k1, u)
    k3 = F(x + dt/2 * k2, u)
    k4 = F(x + dt * k3, u);
    return x+dt/6*(k1 +2*k2 +2*k3 +k4)

In [ ]:
def integrate_euler(x_0, u, dt):
    x = [x_0,]
    for ii in range(len(u)):
        x_i = euler_step(x[-1], u[ii], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
def integrate_rk4(x_0, u, dt):
    x = [x_0,]
    for ii in range(len(u)):
        x_i = rk4_step(x[-1], u[ii], dt)
        x.append(x_i)
    return horzcat(*x).T

In [ ]:
def plot_results_sim(arr, max_par, N):
    arr = np.array(arr)
    timescale_x = np.linspace(0, sol.value(T), N+1)
    timescale_u = np.linspace(0, sol.value(T), N)
    arr_u = sol.value(U)
    plt.figure(figsize=[10,7])
    plt.plot(timescale_x,arr[:,0], label = '$x$')
    plt.plot(timescale_x,arr[:,1], label = "$x'$")
    plt.plot(timescale_u,arr_u[:], label = 'u')
    plt.plot(timescale_x,2+cos(arr[:,0]),':', label = '$2+cos(x)$')
    cross_points = timecrossings(arr[:,1], timescale_x, arr[:,0])
    plt.hlines([0,pi, -pi], 0, timescale_u[-1], 'k', 'dotted')
    plt.hlines([-max_par,max_par], 0, timescale_u[-1], 'r', 'dotted')
    plt.vlines(cross_points, -pi, pi, 'k', 'dotted')
    print(f'Max par: {max_par}, number of crossing points: {len(cross_points)}')
    plt.legend()

### Usando Euler:

In [ ]:
xx = integrate_euler(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(xx, sol.value(max_par), N)

### Usando Runge Kutta

In [ ]:
xx = integrate_rk4(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(xx, sol.value(max_par), N)

## Otras comprobaciones

In [ ]:
N = 300

k1 = F(x, u);
k2 = F(x + dt/2 * k1, u)
k3 = F(x + dt/2 * k2, u)
k4 = F(x + dt * k3, u);
new_x_expr = x+dt/6*(k1 +2*k2 +2*k3 +k4)

new_x_expr = x + dt * F(x, u)
new_x = Function('New_x', [x, u, dt], [new_x_expr])

opti = Opti()
opti.solver('ipopt')

X = opti.variable(N+1,2)
U = opti.variable(N)
T = opti.variable()
u_m = opti.parameter()
#t_m = opti.parameter()

#cost = sum1(2+cos(X[:,0]))*T #**2
cost = T
#cost = -sum1(X[:,0])
opti.minimize(cost)

opti.subject_to(X[0,:].T == [0, 0])
opti.subject_to(cos(X[-1,0]) < -0.9999)
#opti.subject_to(T < t_m)
opti.subject_to(opti.bounded(-0.001,X[-1,1],0.001))

for ii in range(N):
    opti.subject_to(X[ii+1,:].T == new_x(X[ii,:], U[ii], T/N))
    opti.subject_to(opti.bounded(-u_m,U[ii],u_m))

In [ ]:
opti.set_initial(X[:,0], np.linspace(0, pi, N+1))
opti.set_initial(X[:,1], pi/N)
opti.set_initial(T, 50)
max_par = 0.1
opti.set_value(u_m, max_par)

sol = opti.solve()

In [ ]:
plot_results(sol, max_par, N)

In [ ]:
xx = integrate_euler(DM([0,0]), sol.value(U), sol.value(T)/N)
plot_results_sim(xx, sol.value(max_par), N)

In [ ]:
anim = create_anim(sol)

In [ ]:
HTML(anim.to_jshtml())

In [ ]:
results = []
for ii in [0.5, 0.2, 0.1, 0.05, 0.02, 0.01]:
    opti.set_value(u_m, ii)
    est_t = (2 + sqrt(1/ii))*8
    opti.set_initial(T, est_t)
    try:
        sol = opti.solve()
    except:
        pass
    else:
        results.append([sol, ii, N])

In [ ]:
for res in results:
    plot_results(*res)

In [ ]:
xx = np.array([1,0.5, 0.2, 0.1, 0.05, 0.02])
yy = np.array([1,2,3,5,6,8])

plt.plot(1/xx,yy)
#pp = np.arange(50)
pp = 1/xx
plt.plot(pp,1 + np.sqrt(pp))

In [ ]:
opti.debug.show_infeasibilities()